# Purpose
This notebook trains the TrainerGCN using the provided dataset

### Imports and Setup

In [1]:
cd .. 

d:\GitHub-Projects\training-NN-with-Graph-Learning


In [2]:
# Downloaded libraries
import torch
from torch import nn
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T


# Local files
from datasets.graph_data import NNDataset
from models.graph import TrainerGCN

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Debugging Settings
torch.set_printoptions(threshold=12500)

In [4]:
# Constants
TRAINING_SPLIT = 0.8

In [5]:
# Hyperparameters
num_epoch = 8
batch_size = 8

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Data Loading

In [7]:
transform = None
# transform = T.Compose([T.ToUndirected()])

In [9]:
nndataset = NNDataset(root="./", transform=transform)

size = len(nndataset)
train_num = int(size * TRAINING_SPLIT)
test_num = size - train_num

print(
    f"Dataset loaded, {train_num} training samples and {test_num} testing samples")

Dataset loaded, 1120 training samples and 280 testing samples


In [10]:
# Preview of the Data

data = nndataset[0]
data

Data(design=[3], edge_index=[2, 40], x=[13, 503], edge_weight=[40, 1], y_node=[13, 1], y_edge=[40, 1], input_mask=[13, 1], num_nodes=13)

In [11]:
data.is_undirected()

False

In [12]:
train_loader = DataLoader(
    dataset=nndataset[:train_num], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=nndataset[train_num:], batch_size=test_num, shuffle=True)

In [13]:
print("Number of batches:", int(train_num / batch_size))

Number of batches: 140


### Loading the Model

In [14]:
model = TrainerGCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
loss_fn = nn.MSELoss()

In [15]:
model

TrainerGCN(
  (conv1): GATConv(503, 128, heads=1)
  (conv2): GATConv(128, 128, heads=1)
  (dense_1B): Linear(in_features=128, out_features=1, bias=True)
  (dense_1W): Linear(in_features=128, out_features=1, bias=True)
)

### Training and Evaluation

In [16]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for i, data in enumerate(dataloader):
        data.to(device)

        # forward propagation
        out_w, out_b = model(data)
        loss = 0
        loss += loss_fn(out_b, data.y_node)
        loss += loss_fn(out_w, data.y_edge)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print status every n batches
        if i % 35 == 0:
            loss, current = loss.item(), i * batch_size
            print(
                f"Training Loss: {loss:>7f}  [{current:>5d}/{train_num:>5d}]")

In [17]:
def test(dataloader, model, loss_fn):
    model.eval()
    with torch.no_grad():
        data = iter(dataloader).next().to(device)

        # forward propagation
        out_w, out_b = model(data)
        loss = loss_fn(out_b, data.y_node) + loss_fn(out_w, data.y_edge)

        loss = loss.item()
        print(f"Validation Loss: {loss:>7f}")


In [18]:
# Model Training
for epoch in range(num_epoch):
    print(f"Epoch {epoch + 1} / {num_epoch}:")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)

Epoch 1 / 8:
Training Loss: 0.296614  [    0/ 1120]
Training Loss: 0.216534  [  280/ 1120]
Training Loss: 0.188380  [  560/ 1120]
Training Loss: 0.198405  [  840/ 1120]
Validation Loss: 0.299190
Epoch 2 / 8:
Training Loss: 0.145682  [    0/ 1120]
Training Loss: 0.153649  [  280/ 1120]
Training Loss: 0.166315  [  560/ 1120]
Training Loss: 0.164751  [  840/ 1120]
Validation Loss: 0.189213
Epoch 3 / 8:
Training Loss: 0.192565  [    0/ 1120]
Training Loss: 0.137330  [  280/ 1120]
Training Loss: 0.129178  [  560/ 1120]
Training Loss: 0.130158  [  840/ 1120]
Validation Loss: 0.117470
Epoch 4 / 8:
Training Loss: 0.125586  [    0/ 1120]
Training Loss: 0.115377  [  280/ 1120]
Training Loss: 0.183392  [  560/ 1120]
Training Loss: 0.133694  [  840/ 1120]
Validation Loss: 0.144282
Epoch 5 / 8:
Training Loss: 0.117384  [    0/ 1120]
Training Loss: 0.116269  [  280/ 1120]
Training Loss: 0.136261  [  560/ 1120]
Training Loss: 0.135745  [  840/ 1120]
Validation Loss: 0.134949
Epoch 6 / 8:
Training Los

### Comparison Using One Instance of Data

In [19]:
data = nndataset[0]
data = data.to(device)
data

Data(design=[3], edge_index=[2, 40], x=[13, 503], edge_weight=[40, 1], y_node=[13, 1], y_edge=[40, 1], input_mask=[13, 1], num_nodes=13)

In [20]:
data.design

[3, 8, 2]

In [21]:
out_w, out_b = model(data)

In [22]:
out_w

tensor([[ 0.0274],
        [ 0.0265],
        [ 0.0219],
        [ 0.0205],
        [ 0.0271],
        [ 0.0245],
        [ 0.0194],
        [ 0.0283],
        [-0.0364],
        [-0.0373],
        [-0.0418],
        [-0.0432],
        [-0.0367],
        [-0.0393],
        [-0.0443],
        [-0.0354],
        [-0.0287],
        [-0.0295],
        [-0.0341],
        [-0.0354],
        [-0.0289],
        [-0.0316],
        [-0.0365],
        [-0.0277],
        [-0.0060],
        [ 0.0004],
        [-0.0069],
        [-0.0005],
        [-0.0115],
        [-0.0051],
        [-0.0128],
        [-0.0064],
        [-0.0063],
        [ 0.0001],
        [-0.0089],
        [-0.0025],
        [-0.0139],
        [-0.0075],
        [-0.0051],
        [ 0.0013]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [23]:
data.y_edge

tensor([[ 0.2277],
        [-0.5283],
        [ 0.0862],
        [-0.3806],
        [ 1.1312],
        [-0.5555],
        [-0.4393],
        [-0.4372],
        [ 1.0530],
        [ 0.4600],
        [-0.4602],
        [-0.3724],
        [-0.6324],
        [-0.3388],
        [-0.4321],
        [-0.0846],
        [ 0.3264],
        [-0.4622],
        [ 0.4405],
        [ 0.3594],
        [ 0.1728],
        [-1.1223],
        [ 0.6651],
        [ 0.2089],
        [-0.6755],
        [ 0.5243],
        [-0.4433],
        [ 0.1207],
        [ 0.5847],
        [-0.5651],
        [-0.0793],
        [-0.0928],
        [ 0.5919],
        [-0.6201],
        [-0.1842],
        [ 0.6789],
        [ 0.5201],
        [-0.5449],
        [ 0.1860],
        [ 0.1699]], device='cuda:0')

In [24]:
torch.save(model.state_dict(), "./trained_model/gnnmodel")
print("Model saved")

FileNotFoundError: [Errno 2] No such file or directory: '../trained_model/gnnmodel'